In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR OPENAI API KEY"

In [ ]:
!pip install -q langchain-openai langchain-core requests

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [ ]:
# 1. tool creation

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [ ]:
print(multiply.invoke({'a':3, 'b':4}))

In [ ]:
multiply.name

In [ ]:
multiply.description

In [ ]:
multiply.args

In [ ]:
llm = ChatOpenAI()

In [ ]:
llm.invoke('hi')

In [ ]:
# 2. tool binding
llm_with_tools = llm.bind_tools([multiply])

In [ ]:
llm_with_tools.invoke('Hi how are you')

In [ ]:
query = HumanMessage('can you multiply 3 with 1000')

In [ ]:
messages = [query]

In [ ]:
messages

In [ ]:
# 3. tool calling(NO EXECUTION) is done here if LLM decide to use a tool
result = llm_with_tools.invoke(messages)  # LLM just tells which tool to call and with what arguments, it doesn't execute the tool

In [ ]:
messages.append(result)

In [ ]:
messages

In [ ]:
# 4. tool execution (by programmer not LLM)
tool_result = multiply.invoke(result.tool_calls[0])

In [ ]:
tool_result

In [ ]:
messages.append(tool_result)

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages).content

## Currency Conversion Tools

In [ ]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [ ]:
convert.args

In [ ]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

In [ ]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

In [ ]:
llm = ChatOpenAI()

In [ ]:
# tool binding with LLM
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [ ]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [ ]:
messages

In [ ]:
ai_message = llm_with_tools.invoke(messages)

In [ ]:
messages.append(ai_message)

In [ ]:
ai_message.tool_calls

In [ ]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages).content